# My Update
In this notebook I will expand the functionalities of the current RAG system implemented together with my group.

In [3]:
from rag_service.rag_lib import rag_interaction

## Knowledge Base
Instead of getting the Knowledge Base data through the API, as we did for the project, I downloaded the data so I could have it locally and not have to log in every time to get it.

In [ ]:
# TODO: explain the data that there is in the KB
# response_data_list = [kpi_data, machine_data, prediction_data]